# REST API
<img src='pics/0.png' />

## Request objects

<img src='pics/Screenshot from 2018-03-07 15-38-28.png' />

REST framework introduces a Request object that extends the regular HttpRequest, and provides more flexible request parsing. The core functionality of the Request object is the request.data attribute, which is similar to request.POST, but more useful for working with Web APIs.

In [ ]:
request.POST  # Only handles form data.  Only works for 'POST' method.
request.data  # Handles arbitrary data.  Works for 'POST', 'PUT' and 'PATCH' methods.

## Response objects

REST framework also introduces a Response object, which is a type of TemplateResponse that takes unrendered content and uses content negotiation to determine the correct content type to return to the client.

In [ ]:
return Response(data)  # Renders to content type as requested by the client.

## Status codes

Using numeric HTTP status codes in your views doesn't always make for obvious reading, and it's easy to not notice if you get an error code wrong. REST framework provides more explicit identifiers for each status code, such as HTTP_400_BAD_REQUEST in the status module. It's a good idea to use these throughout rather than using numeric identifiers.

## Wrapping API views

REST framework provides two wrappers you can use to write API views.

1. The @api_view decorator for working with function based views.

2. The APIView class for working with class-based views.

These wrappers provide a few bits of functionality such as making sure you receive Request instances in your view, and adding context to Response objects so that content negotiation can be performed.

The wrappers also provide behaviour such as returning 405 Method Not Allowed responses when appropriate, and handling any ParseError exception that occurs when accessing request.data with malformed input.

## @api_view

In [ ]:
from django.http import HttpResponse

from rest_framework import generics
from rest_framework import status
from rest_framework.decorators import api_view
from rest_framework.response import Response

from . import serializers
from my_app import models


@api_view(['GET', 'POST'])
def author_list(request):
    """
    List all code Authors, or create a new author.
    """
    if request.method == 'GET':
        authors = models.Author.objects.all()
        serializer = serializers.AuthorSerializer(authors, many=True)
        return Response(serializer.data)

    elif request.method == 'POST':
        serializer = serializers.AuthorSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)


@api_view(['GET', 'PUT', 'DELETE'])
def author_detail(request, pk):
    """
    Retrieve, update or delete a code author.
    """
    try:
        author = models.Author.objects.get(pk=pk)
    except models.Author.DoesNotExist:
        return Response(status=status.HTTP_404_NOT_FOUND)

    if request.method == 'GET':
        serializer = serializers.AuthorSerializer(author)
        return Response(serializer.data)

    elif request.method == 'PUT':
        serializer = serializers.AuthorSerializer(author, data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

    elif request.method == 'DELETE':
        author.delete()
        return Response(status=status.HTTP_204_NO_CONTENT)


## urls

In [ ]:
from django.conf.urls import url

from rest_framework_swagger.views import get_swagger_view

from . import views

swagger = get_swagger_view(title='Public API')

urlpatterns = [

    url(
        r'^author/$',
        view=views.author_list,
        name='author-list-api-view'
    ),

    url(
        r'^author/(?P<pk>[0-9]+)$',
        views.author_detail,
        name='author_detail-api-view'
    ),

    url(r'^swagger/$', view=swagger, name='swagger'),
]


<img src='pics/Screenshot from 2018-03-07 13-51-26.png' />

## APIView

In [ ]:
from django.http import HttpResponse, Http404

from rest_framework import status, generics
from rest_framework.decorators import api_view
from rest_framework.response import Response
from rest_framework.views import APIView

from . import serializers
from my_app import models

class AuthorList(generics.GenericAPIView):
    """
    List all authors, or create a new Author.
    """

    serializer_class = serializers.AuthorSerializer
    
    def get(self, request, format=None):
        authors = models.Author.objects.all()
        serializer = serializers.AuthorSerializer(authors, many=True)
        return Response(serializer.data)

    def post(self, request, format=None):
        serializer = serializers.AuthorSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)


class AuthorDetail(generics.GenericAPIView):
    """
    Retrieve, update or delete a author instance.
    """

    serializer_class = serializers.AuthorSerializer

    def get_object(self, pk):
        try:
            return models.Author.objects.get(pk=pk)
        except models.Author.DoesNotExist:
            raise Http404

    def get(self, request, pk, format=None):
        author = self.get_object(pk)
        serializer = serializers.AuthorSerializer(author)
        return Response(serializer.data)

    def put(self, request, pk, format=None):
        snippet = self.get_object(pk)
        serializer = serializers.AuthorSerializer(snippet, data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

    def delete(self, request, pk, format=None):
        author = self.get_object(pk)
        author.delete()
        return Response(status=status.HTTP_204_NO_CONTENT)


## urls

In [ ]:
from django.conf.urls import url

from rest_framework_swagger.views import get_swagger_view

from . import views

swagger = get_swagger_view(title='Public API')

urlpatterns = [

    url(
        r'^author/$',
        view=views.AuthorList.as_view(),
        name='author-list-api-view'
    ),

    url(
        r'^author/(?P<pk>[0-9]+)$',
        views.AuthorDetail.as_view(),
        name='author_detail-api-view'
    ),

    url(r'^swagger/$', view=swagger, name='swagger'),
]


<img src='pics/Screenshot from 2018-03-07 14-32-44.png' />